In [1]:
%matplotlib inline

from __future__ import division
import numpy as np
import glob
from astropy import units as u
from astropy.coordinates import SkyCoord
#import healpy as hp
import matplotlib.pyplot as plt
import plot2
from matplotlib.ticker import Formatter, FixedFormatter, FixedLocator
from matplotlib import text
import ephem


ImportError: No module named basemap

In [ ]:
# Import data from a text file (using RA and DEC in two columns)

# Give the correct path to your data file:
psr_positions = np.loadtxt('/Users/Zach/Desktop/nanograv_plots/nanograv_sources_11-2017.txt', usecols=(0,1))

c = SkyCoord(ra=psr_positions[:,0]*u.degree, dec=psr_positions[:,1]*u.degree, frame='icrs')

print c.ra.radian, c.dec.radian


In [ ]:
#define galactic plane function

def plot_mwd(RA,Dec, org = 0, projection='astro mollweide'):
    ''' RA, Dec are arrays of the same length.
    RA takes values in [0,360), Dec in [-90,90],
    which represent angles in degrees.
    org is the origin of the plot, 0 or a multiple of 30 degrees in [0,360).
    title is the title of the figure.
    projection is the kind of projection: 'mollweide', 'aitoff', 'hammer', 'lambert'
    '''
    x = np.remainder(RA+360-org,360) # shift RA values
    ind = x>180
    x[ind] -=360    # scale conversion to [-180, 180]
    x=-x    
    x = SkyCoord(ra=RA*u.degree, dec=Dec*u.degree, frame='icrs')# reverse the scale: East to the left
    #tick_labels = np.array([150, 120, 90, 60, 30, 0, 330, 300, 270, 240, 210])
    #tick_labels = np.remainder(tick_labels+360+org,360)
#     fig = plt.figure(figsize=(10, 5))
#     ax = fig.add_subplot(111, projection=projection, axisbg ='LightCyan')
    ax.plot(x.ra.radian,x.dec.radian)  # convert degrees to radians
#     ax.set_xticklabels(tick_labels)     # we add the scale on the x axis
#     ax.set_title(title)
#     ax.title.set_fontsize(15)
#     ax.set_xlabel("RA")
#     ax.xaxis.label.set_fontsize(12)
#     ax.set_ylabel("Dec")
#     ax.yaxis.label.set_fontsize(12)
#     ax.grid(True)

In [ ]:
# plotting function stolen from bayestar; modified so axes are sent back 

def outline_text(ax):
    """If we are using a new enough version of matplotlib, then
    add a white outline to all text to make it stand out from the background."""
    try:
        # Try to import matplotlib.patheffects (requires matplotlib 1.0+).
        from matplotlib import patheffects
    except ImportError:
        # If import failed, print a warning and do nothing.
        warnings.warn("This version of matplotlib does not support path effects.")
    else:
        # Otherwise, add the path effects.
        effects = [patheffects.withStroke(linewidth=2, foreground='w', zorder=2)]
        for artist in ax.findobj(text.Text):
            artist.set_path_effects(effects)
            

            

In [ ]:
# plotting parameters to gussy things up a li'l

def figsize(scale):
    fig_width_pt = 513.17 #469.755    # Get this from LaTeX using \the\textwidth
    inches_per_pt = 1.0 / 72.27         # Convert pt to inch
    golden_mean = (np.sqrt(5.0)-1.0)/2.0    # Aesthetic ratio
    fig_width = fig_width_pt * inches_per_pt * scale  # width in inches
    fig_height = fig_width * golden_mean              # height in inches
    fig_size = [fig_width, fig_height]
    return fig_size
            

plt.rcParams.update(plt.rcParamsDefault)
params = {'backend': 'pdf',
    'axes.labelsize': 10,
        'lines.markersize': 4,
        'font.size': 10,
        'xtick.major.size':6,
        'xtick.minor.size':3,
        'ytick.major.size':6,
        'ytick.minor.size':3,
        'xtick.major.width':0.5,
        'ytick.major.width':0.5,
        'xtick.minor.width':0.5,
        'ytick.minor.width':0.5,
        'lines.markeredgewidth':1,
        'axes.linewidth':1.2,
        'legend.fontsize': 7,
        'xtick.labelsize': 10,
        'ytick.labelsize': 10,
        #'savefig.dpi':200,
        #'path.simplify':True,
        #'font.family': 'serif',
        #'font.serif':'Times',
        #'text.latex.preamble': [r'\usepackage{amsmath}'],
        #'text.usetex':True,
        'figure.figsize': figsize(1)}

#plt.rcParams.update(params)

In [ ]:
# create projection
ax = plt.subplot(111, projection='astro mollweide')
ax.grid()

# this makes text easier to see on dark colors 
#plot2.outline_text(ax)

# flip map
#h_upper_f = h_upper[flipid]


#add labels
ax.set_xlabel("RA")
ax.set_ylabel("Dec")


#add galactic plane
lon_array = np.arange(0,360)
lat = 0.
eq_array = np.zeros((360,2))
for lon in lon_array:
    ga = ephem.Galactic(np.radians(lon), np.radians(lat))
    eq = ephem.Equatorial(ga)
    eq_array[lon] = np.degrees(eq.get())
RA = eq_array[:,0]
Dec = eq_array[:,1]

plot_mwd(RA, Dec, 360)


# add pulsars, configure marker shapes
ax.plot(c.ra.radian,c.dec.radian, 
        '.', color='b', markersize=10, mew=1.5, mec='k')

# change title of the plot
plt.title('NANOGrav Source Projection')
#plt.suptitle(r'95\% Characteristic Strain Upper Limit, $h_{95}$', y=0.05)
#plt.colorbar(orientation='horizontal', shrink=0.5)
plt.show()

#plt.savefig('/Users/Zach/Desktop/nanograv_plots/source_projection_plot_1.pdf',bbox_inches='tight',dpi=400)